# Option Strategy Demo Notebook

In this notebook, we show how to use the option strategy API to create a strategy and value it.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from serenity_sdk.renderers.derivatives.widget_tools import OptionChooser
from serenity_sdk.renderers.derivatives import market_data
from serenity_sdk.renderers.shared.utils import make_df_from_list
import pandas as pd
import numpy as np

from serenity_types.pricing.derivatives.options.valuation import AssetValuation, StrategyValuationRequest
from serenity_types.refdata.asset import Asset

from serenity_sdk.renderers.derivatives.option_finders import OptionFinder
# create an alias to the api
api = connect_widget.get_api()

## Setup

In [ ]:
# Set the datetime
as_of_datetime = pd.Timestamp.now(tz='UTC').to_pydatetime()
as_of_datetime

In [ ]:
# Find the supported underliers
underlier_map = make_df_from_list(api.pricer().get_supported_underliers()).set_index('native_symbol')['asset_id']
underlier_map

In [ ]:
# Find exchanges
# use deribit, bybit
exchange_map = api.refdata().get_exchanges()
exchange_map

In [ ]:
# select an exchange and underlier
exchange_id = exchange_map['deribit']
underlier_id = underlier_map['BTC']
print(f"exchange: {exchange_id}, underlier: {underlier_id}")

In [ ]:
# get the spot price for the underlier so that reasonable strikes can be chosen
spot_price = market_data.get_spot_price(api, underlier_id)

## Initialize the option finder
This helps to select options for the strategy.

In [ ]:
option_finder = OptionFinder(api, exchange_id, underlier_id)

In [ ]:
available_expiries = option_finder.expiry_datetimes
available_strikes = option_finder.strikes
print(f"expiries: {available_expiries}")
print(f"strikes: {available_strikes}")

## Call Spread

In [ ]:
# select an expiry and look up available options for the expiry
time_to_expiry = available_expiries[-2]
options_found = option_finder.get_options_for_expiry(time_to_expiry, 0.9*spot_price, 1.1*spot_price)
options_found

In [ ]:
# Set up a strategy and compute the valuation
strategy_request = StrategyValuationRequest(
    legs=[
        AssetValuation(
            valuation_id='call low strike',
            asset_id=options_found['CALL'].iloc[0],
            qty = 1
        ),
        AssetValuation(
            valuation_id='call high strike',
            asset_id=options_found['CALL'].iloc[-1],
            qty = -1
        )
    ]
)

api.pricer().compute_strategy_valuation(
    request=strategy_request
)

## Calendar spread

In [ ]:
# Find a strike for calendar spread
# Get the strike closest to the spot price where there are at least 4 options available so that 
# a calendar spread can be constructed
strikes_by_distance_to_spot = sorted(available_strikes, key=lambda x: np.abs(x - spot_price))
for strike_price in strikes_by_distance_to_spot:
    options_found = option_finder.get_options_for_strike(strike_price)
    if len(options_found) > 4:
        break
calendar_strike = strike_price
options_found

In [ ]:
# Set up a strategy and compute the valuation
strategy_request = StrategyValuationRequest(
    legs=[
        AssetValuation(
            valuation_id='call short expiry',
            asset_id=options_found['CALL'].iloc[-2],
            qty = 1
        ),
        AssetValuation(
            valuation_id='call long expiry',
            asset_id=options_found['CALL'].iloc[-1],
            qty = -1
        )
    ]
)

api.pricer().compute_strategy_valuation(
    request=strategy_request
)